In [4]:
import argparse
import sys
import os
import pickle
import logging
import copy
import signal
import pathos.multiprocessing
import numpy as np
import scipy
from tqdm import trange

sys.path.insert(0, '../../../network')
from network import Population, RateNetwork
from connectivity import SparseConnectivity, LinearSynapse, ThresholdPlasticityRule
from sequences import GaussianSequence

import matplotlib.pyplot as plt
from brokenaxes import brokenaxes

Load data

In [5]:
datapath = "data/data_ab.npy"
state, _ = np.load(datapath, allow_pickle=True)

Compute correlations across days

In [7]:
n_realizations = 1
n_days = 30

def active_idxs(state, threshold=0.0, t=0):
    return np.any(state.T[:,t:] > threshold, axis=1).nonzero()[0]

idxs = active_idxs(state[0][0])
idxs2 = active_idxs(state[0][-1])
corr = np.zeros((n_realizations, n_days, idxs.size))
corr_reverse = np.zeros((n_realizations, n_days, idxs2.size))
for m in range(n_realizations):
    for n in trange(n_days):
        #print("Computing correlation for day", n, "realization", m)
        corr[m, n,:] = np.asarray([
            scipy.stats.pearsonr(state[m][0].T[i], state[m][n].T[i])[0] for i in idxs])
        corr_reverse[m, n,:] = np.asarray([
            scipy.stats.pearsonr(state[m][-1].T[i], state[m][n].T[i])[0] for i in idxs2])

np.save(open("data/data_c1.npy", "wb"), corr)
np.save(open("data/data_c2.npy", "wb"), corr_reverse)

100%|██████████| 30/30 [02:33<00:00,  5.10s/it]
